## Improved FNN by Cao

In [6]:
import os
import argparse
import logging
import time
import numpy as np
import numpy.random as npr
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt


In [7]:
def load_data_normalize(obs_dim, datafilepath):
    datafilepath = 'C:/Users/shiny/Documents/NeuralODE_HumanGait/Humangaitdata.npy'
    data = np.load(datafilepath)
    traj_tot = np.load(datafilepath).reshape(72, 1500, obs_dim)
    traj_tot = traj_tot[:,150:1350,:]
    data = data[:, 300:1200, :]
    data = data.reshape(72, 900, obs_dim)
    noise_std = 0.2

    orig_trajs = np.zeros((data.shape[0],data.shape[1],data.shape[2]))
    for i in range(data.shape[0]):
        for j in range(data.shape[2]):
            trajs = data[i,:,j]
            trajs_tot = traj_tot[i,:,j]
            orig_trajs[i,:,j] = (trajs - trajs_tot.mean()) / trajs_tot.std()
            
    #samp_trajs += npr.randn(*samp_trajs.shape) * noise_std #add noise

    return orig_trajs


In [8]:
samp_trajs = load_data_normalize(6, 'C:/Users/shiny/Documents/GaitControl/All_Rodent_concatenated_csv/All_Rodent_fps200.npy')

In [11]:
X = samp_trajs[:,:,:]

In [12]:
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import numpy2ri

nonlinearTseries = importr("nonlinearTseries")
data = numpy2ri.numpy2rpy(X)

cao_emb_dim = nonlinearTseries.estimateEmbeddingDim(
    data,  # time series
    len(data),  # number of points to use, use entire series
    18,  # time delay
    12,  # max no. of dimension
    0.95,  # threshold value
    0.1,  # max relative change
    True,  # do the plot
    "Computing the embedding dimension",  # main
    "dimension (d)",  # x_label
    "E1(d) & E2(d)",  # y_label
    ro.NULL,  # x_lim
    ro.NULL,  # y_lim
    0  # add a small amount of noise to the original series to avoid the
          # appearance of false neighbours due to discretization errors.
          # This also prevents the method to fail with periodic signals, 0 for no noise
)

embedding_dimension = int(cao_emb_dim[0])

In [13]:
embedding_dimension

-2147483648

In [28]:
plt.savefig('books_read.png')